In [1]:
%pip install openai
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import json
import pandas as pd
import openai
from openai import OpenAI
from tqdm import tqdm

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Your-api-key"

In [19]:
def build_prompt(row):
    return (
        f"Supppose there's a Knapsack instance with number_of_elements={int(row['number_of_elements'])}, "
        f"capacity={row['capacity']}, "
        f"max_weight={row['max_weight']}, min_weight={row['min_weight']}, "
        f"mean_weight={row['mean_weight']}, median_weight={row['median_weight']}, std_weight={row['std_weight']}, "
        f"weight_range={row['weight_range']}, "
        F"max_profit={row['max_profit']}, min_profit={row['min_profit']}, "
        f"mean_profit={row['mean_profit']}, median_profit={row['median_profit']}, std_profit={row['std_profit']}, "
        f"profit_range={row['profit_range']}, "
        f"renting_ratio={row['renting_ratio']}, "
        f"mean_weight_profit_ratio={row['mean_weight_profit_ratio']}, "
        f"median_weight_profit_ratio={row['median_weight_profit_ratio']}, "
        f"capacity_mean_weight_ratio={row['capacity_mean_weight_ratio']}, "
        f"capacity_median_weight_ratio={row['capacity_median_weight_ratio']}, "
        f"capacity_std_weight_ratio={row['capacity_std_weight_ratio']}, "
        f"std_weight_profit_ratio={row['std_weight_profit_ratio']}, "
        f"weight_profit_correlation={row['weight_profit_correlation']}, "
        f"cpu_cores={int(row['cpu_cores'])}, ram={int(row['ram'])}GB. "
        "Based on these features, predict the **solution_time** (in milliseconds), **optimality_gap** (fraction), "
        "**peak_memory** (in KB) if solved using the **branch and bound** algorithm for profit-maximization in 0-1 knapsack. "
        "Return **only** a JSON object like :\n "
        '{"solution_time": <double>, "optimality_gap": <double>, "memory_kb": <int>}.'
    )


In [20]:
def strip_code(text: str) -> str:
    text = text.strip()
    if text.startswith("```") and text.endswith("```"):
        body = "\n".join(text.splitlines()[1:-1])
        return body.strip()
    return text

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

df = pd.read_csv("input.csv")  #Path to the instance-feature file for a specific algorithm



output_path = "output.csv"
cols = df.columns.tolist() + ["solution_time", "optimality_gap", "memory_kb"]
pd.DataFrame(columns=cols).to_csv(output_path, index=False)



for idx, row in tqdm(df.iterrows(), total=len(df)):
    prompt = build_prompt(row)

    try:
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        raw = resp.choices[0].message.content or ""
        clean = strip_code(raw)
        out = json.loads(clean)

        row["solution_time"]  = out["solution_time"]
        row["optimality_gap"] = out["optimality_gap"]
        row["memory_kb"]      = out["memory_kb"]

    except json.JSONDecodeError as e:
        print(f"[Row {idx}] JSON parse error: {e}")
        print(f"[Row {idx}] Response:\n{raw!r}\n")
        row["solution_time"]  = None
        row["optimality_gap"] = None
        row["memory_kb"]      = None

    pd.DataFrame([row])[cols].to_csv(output_path, mode="a", header=False, index=False)
    time.sleep(1)

100%|██████████| 50/50 [01:40<00:00,  2.01s/it]
